<a href="https://colab.research.google.com/github/somewheresy/somewhere-diffusion/blob/main/Somewhere_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Somewhere Diffusion

#### A notebook for training diffusion models on custom datasets with video and photo input, allowing users to synthesize novel images from those models, with Google Drive support and ESRGAN image upscaling.

## Introduction
Various methods for the generation of images using machine learning have rapidly expanded as they have become more accessible through services like [Midjourney](https://www.midjourney.com/home/) and OpenAI's [DALL-E 2](https://openai.com/dall-e-2/). However, controversy surrounding intellectual property rights of artists have created a schism in the creative community over the topic of "AI Art". 

This is in part due to private companies training models for image generation on datasets that may include copyrighted, or even illegal content (overlooked due to the size of the dataset being practically unmoderable, such as the case with LAION-400M) without the consent of the copyright holders.

The intention of this notebook is to provide a tool which can be used for training models on any collection of images, changing the social contract of image generation into a consent-forward proposal. Although this notebook still utilizes OpenAI's CLIP for image-text pair retrieval, it is model-agnostic: allowing users to load whichever .pt model file they please instead of potentially blurring ethical lines of consent and ownership (unless, of course, users deliberately choose to do so).

The purpose of the notebook is to allow traditional artists to automate parts of their process or act as a jumping-off point for them to develop new ways of using this tool, without worrying if they're feeding into a for-profit machine cannibalizing their work for monetary gain they are not receiving a portion of. In summary, the only remaining part of this tool which uses large (and opaque) image datasets will be CLIP -- *the onus will be on them, and not the AI Art community at large*.

## Requirements

At the current point in time, Colab Pro and a GPU with 16GB of RAM or more is needed. Optimizations will be pursued to allow this notebook to run on lower-end GPUs, with the intention of accessibility across the global software community.

## Architecture / How-To

TBD

## Prior Art

This notebook is constructed on methods initially developed by [un1tz3r0](https://linktr.ee/un1tz3r0), which themselves were based on a notebook by [Alex Spirin](https://twitter.com/devdef). The methods that are used for image generation using diffusion models largely are to the credit of [Katherine Crowson](https://twitter.com/RiversHaveWings) and [Advadnoun](https://twitter.com/advadnoun) who helped to pioneer the method of using CLIP and VQ-GAN together for image synthesis.

The portion of this notebook which synthesizes images is adapted from my prior notebook, the [S2ML Image Generator](https://github.com/somewheresy/S2ML-Generators/blob/main/S2ML_Image_Generator.ipynb).
## Research

The end-state of this project will be a continuously repeatable research methodology for evaluating the subjective fidelity of this notebook, as well as perhaps a method for quantitatively validating the proximity of output images to the CLIP-embeddings in latent space. The research will live in the notebook as a means to foster accessibility. 



In [ ]:
#@markdown #Check GPU/VRAM & RAM

#@markdown ####If you receive an error that NVIDIA-SMI has failed, you will have to enable the GPU in the Notebook settings. Click on **Edit > Notebook Settings** and select GPU as hardware accelerator.

from psutil import virtual_memory
!nvidia-smi --query-gpu=gpu_name,gpu_bus_id,vbios_version --format=csv
gpu_name = !nvidia-smi --query-gpu=gpu_name, --format=csv

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))


name, pci.bus_id, vbios_version
Tesla P100-PCIE-16GB, 00000000:00:04.0, 86.00.52.00.02
Your runtime has 13.6 gigabytes of available RAM



In [ ]:
#@title Connect Google Drive
import os

## from https://svn.blender.org/svnroot/bf-blender/trunk/blender/build_files/scons/tools/bcolors.py
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

from google.colab import drive
drive.mount('/content/drive')
root_path = "/content/drive"

def checkRootPath():
    if len(root_path) > 0:
        os.chdir(root_path) # Changes directory to absolute root path
        print(bcolors.BOLD + "Root Path Check: " + bcolors.OKBLUE)
        !pwd

checkRootPath()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Root Path Check: 
/content/drive


In [ ]:
#@title Make a workspace for Somewhere Diffusion, a project folder & set path to that folder
#@markdown #### In the case the project folder already exists, you can use this block to set the directory to a previous project.
workspace_name = "Somewhere Diffusion" #@param {type: "string"}
root_path = "/content"
if len(workspace_name) > 0:
    path_tmp = root_path + "/drive/MyDrive/" + workspace_name
    if not os.path.exists(path_tmp):
        os.mkdir(path_tmp)
        print("Created folder & set root path to: " + root_path)
    root_path = path_tmp
print("Work & set root path to: " + root_path)
project_name = "Notebook Development" #@param {type: "string"}
if len(project_name) > 0:
      path_tmp = root_path + "/" + project_name
      if not os.path.exists(path_tmp):
          os.mkdir(path_tmp)
      _path = path_tmp
print(bcolors.BOLD + "Created project subfolder & set root path to: " + bcolors.OKBLUE + _path)

Created folder & set root path to: /content
Created project subfolder & set root path to: /content/drive/MyDrive/Somewhere Diffusion/Notebook Development
